Below is the code for calculationg ENPV with a modified method, using uniform distribution to select the initial injection rates. we have 8 arrays for 8 wells and they have different injection rates. we have 30 initial injection rates. 

In [1]:
# import necessary packages

from ecl.summary import EclSum
import sys
import os
import numpy as np
from array import *
import pandas as pd
import GPy
import GPyOpt
from GPyOpt.methods import BayesianOptimization
import matplotlib.pyplot as plt

In [2]:
# parameters that we need to calculate ENPV

OP =  300   #Oil Price $ per m3
WPP = 47.5    #Water Production Price $ per m3
WIP = 12.5  #WaterInjection Price $ per m3
Study_period = np.arange(10) #Years
DiscountRate = 0.08
NPV = []
Inj_rate_Opt = []
NPV_Opt = []

In [ ]:
#initial random injection rates selection using uniform distribution

Inj_Data = {'U1' : np.random.uniform(low=5, high=100, size=(30)),
            'U2' : np.random.uniform(low=5, high=100, size=(30)),
            'U3' : np.random.uniform(low=5, high=100, size=(30)),
            'U4' : np.random.uniform(low=5, high=100, size=(30)),
            'U5' : np.random.uniform(low=5, high=100, size=(30)),
            'U6' : np.random.uniform(low=5, high=100, size=(30)),
            'U7' : np.random.uniform(low=5, high=100, size=(30)),
            'U8' : np.random.uniform(low=5, high=100, size=(30))}

InjRate_NPV_df=pd.DataFrame(Inj_Data)
InjRate = InjRate_NPV_df.T
display(InjRate_NPV_df)

Inject_Different_Rates=InjRate_NPV_df.to_numpy()
Inject_Different_Rates=np.around(Inject_Different_Rates,2)


In [4]:
# Modify Injection Rates with different Injection Rates 

def modify_Different_InjRate(Init_InjRate):
    
    Schedule_NEW_sample=pd.read_csv('SCHEDULE_NEW_sample.INC')
    #print("InjRate=" , Init_InjRate)

    Schedule_NEW_sample['WCONINJE'][0] = Schedule_NEW_sample['WCONINJE'][0].replace('U1' , str(Init_InjRate[0]))
    Schedule_NEW_sample['WCONINJE'][1] = Schedule_NEW_sample['WCONINJE'][1].replace('U2' , str(Init_InjRate[1]))
    Schedule_NEW_sample['WCONINJE'][2] = Schedule_NEW_sample['WCONINJE'][2].replace('U3' , str(Init_InjRate[2]))
    Schedule_NEW_sample['WCONINJE'][3] = Schedule_NEW_sample['WCONINJE'][3].replace('U4' , str(Init_InjRate[3]))
    Schedule_NEW_sample['WCONINJE'][4] = Schedule_NEW_sample['WCONINJE'][4].replace('U5' , str(Init_InjRate[4]))
    Schedule_NEW_sample['WCONINJE'][5] = Schedule_NEW_sample['WCONINJE'][5].replace('U6' , str(Init_InjRate[5]))
    Schedule_NEW_sample['WCONINJE'][6] = Schedule_NEW_sample['WCONINJE'][6].replace('U7' , str(Init_InjRate[6]))
    Schedule_NEW_sample['WCONINJE'][7] = Schedule_NEW_sample['WCONINJE'][7].replace('U8' , str(Init_InjRate[7]))

    Schedule_NEW_sample.to_csv('SCHEDULE_NEW.INC',index=False)



In [5]:
# NPV calculation function

def NPV_Calc():
    ''' In this function, we need to read the data provided by Eclips simulator,
    then, Eclips provides data for 10 years with intervals of 30 days, I considered every 360 days as a year, 
    since the discount rate is provided as yearly basis, I chose yearly data, calculated net cash flows for evey year,
    discounted yearly and summed up all the discounted cash flows to calculate the NPV,
    This is done for every realizations'''
    
    summary = EclSum("ENSEMBLE_1.UNSMRY")
    Time = summary.numpy_vector("TIME")
    Years = summary.numpy_vector("YEARS")
    FOE = summary.numpy_vector("FOE")
    FOPR = summary.numpy_vector("FOPR")
    FOPT = summary.numpy_vector("FOPT")
    FPR = summary.numpy_vector("FPR")
    FWCT = summary.numpy_vector("FWCT")
    FWIR = summary.numpy_vector("FWIR")
    FWPR = summary.numpy_vector("FWPR")
    FWPT = summary.numpy_vector("FWPT")

    df = pd.DataFrame(list(zip(Time, Years, FOE, FOPR, FOPT,FPR, FWCT, FWIR, FWPR, FWPT)), 
    columns=["Time", "Years", "FOE","FOPR", "FOPT","FPR","FWCT", "FWIR","FWPR","FWPT"])
    df2=df[['Time','FOPT','FWPT', 'FWIR']]
    df2=df2.iloc[11::12]
    df2.reset_index(drop=True, inplace=True)
    NetCashFlows = OP * np.array(df2['FOPT']) - WPP * np.array(df2['FWPT']) - WIP * (np.array(df2['FWIR'])*360)
    DiscountFactors_end = 1/(1+DiscountRate)**Study_period
    DiscountedNetCashFlows = NetCashFlows*DiscountFactors_end
    NPV_end = sum(DiscountedNetCashFlows)/6.29
    NPV.append(NPV_end)
    #print("NPV_end for realization: " ,  NPV_end)
    return NPV
    


In [11]:
# NPV function which will be passed to BO

def NPV_BO(X_init):
    '''In this function, we will calculate the ENPV, first we need to open each realization's directory, 
    change the injection rates acording to what BO suggests, run the simulator, then calculate the NPV for the 
    1st realization, if the NPV calculated in the 1st realization is greater than the average ENPV calculated
    from initial injection rates, other 9 realizations with be run and their NPVs will be calculated, otherwise,
    BO will provide another injection rate'''
    for i in X_init:
        for u in range(1,11):
            os.chdir('/home/mina/Documents/Data/ENPV-Calc/' + str(u))
            modify_Different_InjRate(i)
            os.system(command='flow Ensemble_1.DATA > /dev/null')
            NPV_Calc()
            if u ==1 and NPV < ENPV_avg:
                break
        #print('NPV: ' , NPV)
        ENPV=round(sum(NPV)/len(NPV),4)
        print('ENPV of ' + str(i) , ': ', ENPV)
    NPV.clear()
    #print("NPV_now: " , NPV)
    return ENPV

In [12]:
#initial injection rates and the corresponding ENPVs are read to be passed to BO
X_init = Inject_Different_Rates
Y_init = pd.read_csv('/home/mina/Documents/Data/ENPV-Calc/Y_initial_uniform2.csv', header= None)
Y_init = Y_init.to_numpy()

In [13]:
# I used Bayesian optimization package here I initialize the BO

ENPV_avg=np.average(Y_init)
bounds = np.array([[5, 100]])
k = GPy.kern.RBF(1)
bds = [{'name': 'X', 'type': 'continuous', 'domain': bounds.ravel()},
       {'name': 'X', 'type': 'continuous', 'domain': bounds.ravel()},
       {'name': 'X', 'type': 'continuous', 'domain': bounds.ravel()},
       {'name': 'X', 'type': 'continuous', 'domain': bounds.ravel()},
       {'name': 'X', 'type': 'continuous', 'domain': bounds.ravel()},
       {'name': 'X', 'type': 'continuous', 'domain': bounds.ravel()},
       {'name': 'X', 'type': 'continuous', 'domain': bounds.ravel()},
       {'name': 'X', 'type': 'continuous', 'domain': bounds.ravel()}]

optimizer = BayesianOptimization(f=NPV_BO, 
                                 domain=bds,
                                 model_type='GP',
                                 kernel=k,
                                 acquisition_type ='EI',
                                 X=X_init,
                                 Y = -Y_init,
                                 maximize=True)



In [ ]:
# I run the BO for 20 iterations
optimizer.run_optimization(max_iter=20)

In [10]:
#In below code, I saved the initial random injection rates so I can used them for BO initialzation later
np.savetxt('X_initial_uniform2.csv', optimizer.X, delimiter=',')

In [9]:
#here, I saved the ENPV calculated from initial values, so I can use it for BO initialization later
np.savetxt('Y_initial_uniform2.csv', -optimizer.Y , delimiter=',')

In [ ]:
# best injection rate 
optimizer.x_opt

In [ ]:
# best ENPV
optimizer.fx_opt

In [ ]:
# convergence plot from BO package which I did not use
optimizer.plot_convergence()


In [46]:
# convergence plot function that I used to show the BO convergence
def plot_convergence(Y_sample, n_init=2):
    plt.figure(figsize=(12, 3))

    y = Y_sample[n_init:].ravel()
    r = range(1, len(y)+1)
    
    y_max_watermark = np.maximum.accumulate(y)

    plt.subplot(1, 2, 2)
    plt.plot(r, y_max_watermark, 'ro-')
    plt.xlabel('Iteration')
    plt.ylabel('Best Y')
    plt.title('Value of best selected sample')

In [ ]:
plot_convergence(-optimizer.Y)